In [1]:
home="/frazer01/home/paola/Family1070/private_output/Jaccard"

library(gplots)
library(RColorBrewer)
library(pheatmap)
library(matrixStats)



Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess

matrixStats v0.50.2 (2016-04-24) successfully loaded. See ?matrixStats for help.


In [2]:
setwd(home)
setwd("heatmaps")
ch=read.csv("Chrom_25states.csv")
ch.o=ch[order(ch$STATE.NO.),]
col1=as.character(ch.o$COLOR_NAME)
rownames(ch.o)=ch.o$Name

ch.3=data.frame(ch.o[,3])
colnames(ch.3)="ChromHMM"
rownames(ch.3)=rownames(ch.o)

ChromHMM     <- as.character(ch.o[,4])
names(ChromHMM) <- ch.3[,1]
anno_colors <- list(ChromHMM = ChromHMM)


In [3]:
files=c(
  '/home/paola/Jupiter_jaccard/output/jaccard_analysis_25_state/ATAC_CM_newdata/jaccard.tsv',
  '/home/paola/Jupiter_jaccard/output/jaccard_analysis_25_state/ATAC_CM_meta_macs2/jaccard.tsv',
  '/home/paola/Jupiter_jaccard/output/jaccard_analysis_25_state/ATAC_iPSC_meta_macs2/jaccard.tsv',
  '/projects/CARDIPS/analysis/cardips-pfamily/output/jaccard_analysis_25_state/h3k27ac_cm/jaccard.tsv',
  '/projects/CARDIPS/analysis/cardips-pfamily/output/jaccard_analysis_25_state/h3k27ac_ips/jaccard.tsv',
  '/projects/CARDIPS/analysis/cardips-pfamily/output/jaccard_analysis_25_state/nkx25/jaccard.tsv',
  '/projects/CARDIPS/analysis/cardips-pfamily/output/jaccard_analysis_25_state/srf/jaccard.tsv',
  '/projects/CARDIPS/analysis/cardips-pfamily/output/jaccard_analysis_25_state/gro/jaccard.tsv'
)
names=c("ATAC-Seq_CM_newdata", "ATAC-Seq_CM","ATAC-Seq_iPSC",'H3K27ac_CM','H3K27ac_iPSC',"NKX2-5_CM", "SRF_CM","Gro-Seq_CM"  )


In [4]:
files=list.files("/home/paola/Family1070/private_output/Jaccard/output/jaccard_analysis_25_state")
names = files
names[length(files)]= 'NKX2-5'

In [5]:
names

[1] "ATAC_CM"            "ATAC_IPSC"          "H3K27AC_CM"        
 [4] "H3K27AC_CM_FS003"   "H3K27AC_CM_FS009"   "H3K27AC_CM_FS011"  
 [7] "H3K27AC_CM_UDID01"  "H3K27AC_CM_UDID045" "H3K27AC_IPSC"      
[10] "NKX2-5"

### Method 1: Use normalize using Z-Scores and plot only a subset of the tissues

In [14]:
method_folder="z-score"
dir.create(method_folder)


for (i in 1:length(files)) {

########################
  file=paste( home, "output/jaccard_analysis_25_state", files[i], "jaccard_z_score.tsv", sep="/")
  mat=read.table(file, header=T , row.names=1, check.names=FALSE, stringsAsFactor=FALSE)
  x=mat
  ######################################################
#   #Transform to BH pvalues
#   mat2<-pnorm(as.matrix(abs(mat)),lower.tail=F)*2
#   mat2<-p.adjust(mat2, method="BH")
#   mat2<-matrix(-log(mat2,10), nrow=nrow(mat))
#   mat2<-mat*mat2/abs(mat)
#   colnames(mat2)<-colnames(mat)
#   rownames(mat2)<-rownames(mat)
#   mat<-mat2
#   x=mat2
  ######################################################
  ##### Min
  
  x1=min(x)
  x2=min( x[x!=x1] )
  x3=min( x[x!=x2&x!=x1] )
  x4=min( x[x!=x3&x!=x2&x!=x1] )
  x5=min( x[x!=x4&x!=x3&x!=x2&x!=x1] )
  
  a=which(mat==c(x1), arr.ind=T)
  b=which(mat==c(x2), arr.ind=T)
  c=which(mat==c(x3), arr.ind=T)
  d=which(mat==c(x4), arr.ind=T)
  e=which(mat==c(x5), arr.ind=T)
  State=c(colnames(mat)[a[,2]],colnames(mat)[b[,2]],colnames(mat)[c[,2]],colnames(mat)[d[,2]],colnames(mat)[e[,2]])
  Tissue=c(rownames(a),rownames(b), rownames(c),rownames(d),rownames(e))
  Score=c(x1,x2,x3,x4,x5)
  res=cbind(Tissue, State, Score)
  
  road_tissues=read.csv("EIDlegend.csv", stringsAsFactor=F, check.names=F, header=FALSE)
  
  res=merge(res, road_tissues, by.x=1, by.y=1)
  colnames(res)[4]="Tissue_name"
  write.table(res, paste( method_folder , "/", names[i], ".res.min.txt", sep=""), col.names=NA , sep="\t", quote=F)
  
  ####MAx
  x1=max(x)
  x2=max( x[x!=x1] )
  x3=max( x[x!=x2&x!=x1] )
  x4=max( x[x!=x3&x!=x2&x!=x1] )
  x5=max( x[x!=x4&x!=x3&x!=x2&x!=x1] )
  
  a=which(mat==c(x1), arr.ind=T)
  b=which(mat==c(x2), arr.ind=T)
  c=which(mat==c(x3), arr.ind=T)
  d=which(mat==c(x4), arr.ind=T)
  e=which(mat==c(x5), arr.ind=T)
  State=c(colnames(mat)[a[,2]],colnames(mat)[b[,2]],colnames(mat)[c[,2]],colnames(mat)[d[,2]],colnames(mat)[e[,2]])
  Tissue=c(rownames(a),rownames(b), rownames(c),rownames(d),rownames(e))
  Score=c(x1,x2,x3,x4,x5)
  res=cbind(Tissue, State, Score)
  
  res=merge(res, road_tissues, by.x=1, by.y=1)
  colnames(res)[4]="Tissue_name"
  write.table( res, paste(method_folder , "/",names[i], ".res.max.txt", sep=""), col.names=NA , sep="\t", quote=F)
  
  ########################################################################
  ##### Heatmaps    ######################################################
  
  colnames(mat)=rownames(ch.3)
  
  tissue_list4=c("E002", "E004", "E011", "E012", "E013", "E014", "E020", 'E021', 
                 'E029', 'E031', 'E034', 'E056', 'E059', 'E065', 'E082', 'E083', 'E084',
                 'E090', 'E095', 'E098', 'E104', 'E105', 'E108', 'E113', "E123")
  
  tissue_list=tissue_list4 #final list to use
  
  
  mat=mat[tissue_list,]
  
  road_tissues=read.csv("EIDlegend.csv", stringsAsFactor=F, check.names=F, header=FALSE, row.names=1)
  ti=road_tissues[tissue_list,]
  rownames(mat)=ti
  
  # ###abbreviated names
  # road_tissues=read.csv("../Roadmap_metadata.csv", stringsAsFactor=F, check.names=F, header=TRUE,row.names=1)
  # ti=road_tissues[tissue_list,]
  # rownames(mat)=ti$name
  
color=colorRampPalette(c("dodgerblue3", "white", "orangered3"))(50)
#color=colorRampPalette(c( "white", "dodgerblue3"))(50)
  
  pdf(paste( method_folder , "/", names[i], ".heatmap_jaccard.pdf", sep=""), width=8, height=10)
  pheatmap(mat, show_rownames=TRUE,cluster_rows=T,
           annotation_col=ch.3, color = color,
           cellwidth = 8, cellheight = 8, annotation_colors=anno_colors, annotation_legend=FALSE,
           fontsize=9, border_color=NA, breaks=seq(-abs(max(mat)), abs(max(mat)), length.out=50) )
  
  
    
    
  dev.off()
  }


Warning message in dir.create(method_folder):
“'z-score' already exists”

### Method 2: Mean Center normalize

In [30]:
method_folder="mean-center"
dir.create(method_folder)


for (i in 1:length(files)) {

########################
  file=paste( home, "output/jaccard_analysis_25_state", files[i], "jaccard.tsv", sep="/")
  mat=read.table(file, header=T , row.names=1, check.names=FALSE, stringsAsFactor=FALSE)
  x=mat
      
    colnames(mat)=rownames(ch.3)
    mat<-mat[order(rowMeans(mat), decreasing=T),]
    mat=scale(mat, center = TRUE, scale = FALSE)

  ########################################################################
  ##### Heatmaps    ######################################################
  

  tissue_list=rownames(mat)  
  
  mat=mat[tissue_list,]
  
  road_tissues=read.csv("EIDlegend.csv", stringsAsFactor=F, check.names=F, header=FALSE, row.names=1)
  ti=road_tissues[tissue_list,]
  rownames(mat)=ti
  
  # ###abbreviated names
  # road_tissues=read.csv("Roadmap_metadata.csv", stringsAsFactor=F, check.names=F, header=TRUE,row.names=1)
  # ti=road_tissues[tissue_list,]
  # rownames(mat)=ti$name
  
color=colorRampPalette(c("dodgerblue3", "white", "orangered3"))(50)

  
   pdf(paste( method_folder , "/", names[i], ".heatmap_jaccard_all_tissues_mean_center.pdf", sep=""), width=10, height=20)
  pheatmap(mat, show_rownames=TRUE,cluster_rows=F,
           annotation_col=ch.3, color = color,
           cellwidth = 8, cellheight = 8, annotation_colors=anno_colors, annotation_legend=FALSE,
           fontsize=9, border_color=NA, breaks=seq(-abs(max(mat)), abs(max(mat)), length.out=50) )
  
  
    
    
  dev.off()
  }


Warning message in dir.create(method_folder):
"'mean-center' already exists"

### Method 3: Mean center normalize only enhancers

In [16]:
colnames(mat)

[1] "1_TssA"      "2_PromU"     "3_PromD1"    "4_PromD2"    "5_Tx5"      
 [6] "6_Tx"        "7_Tx3"       "8_TxWk"      "9_TxReg"     "10_TxEnh5"  
[11] "11_TxEnh3"   "12_TxEnhW"   "13_EnhA1"    "14_EnhA2"    "15_EnhAF"   
[16] "16_EnhW1"    "17_EnhW2"    "18_EnhAc"    "19_DNase"    "20_ZNF_Rpts"
[21] "21_Het"      "22_PromP"    "23_PromBiv"  "24_ReprPC"   "25_Quies"

In [29]:
method_folder="enhancers-mean-center"
dir.create(method_folder)


enhancers=c('9_TxReg','10_TxEnh5', '11_TxEnh3', '12_TxEnhW' ,'13_EnhA1' ,'14_EnhA2', '15_EnhAF', '16_EnhW1' ,'17_EnhW2', '18_EnhAc')
#enhancers=c('13_EnhA1' ,'14_EnhA2', '15_EnhAF', '16_EnhW1' ,'17_EnhW2', '18_EnhAc')

for (i in 1:length(files)) {

########################
  file=paste( home, "output/jaccard_analysis_25_state", files[i], "jaccard.tsv", sep="/")
  mat=read.table(file, header=T , row.names=1, check.names=FALSE, stringsAsFactor=FALSE)
  x=mat
    
    colnames(mat)=rownames(ch.3)
    mat<-subset(mat, select=as.character(enhancers))
    mat<-mat[order(rowMeans(mat), decreasing=T),]
    #mat<-mat[order(rowMeans(mat[,c('13_EnhA1' ,'14_EnhA2', '15_EnhAF')]), decreasing=T),]
    mat=scale(mat, center = TRUE, scale = FALSE)

  ########################################################################
  ##### Heatmaps    ######################################################
  

  tissue_list=rownames(mat)  
  
  mat=mat[tissue_list,]
  
  road_tissues=read.csv("EIDlegend.csv", stringsAsFactor=F, check.names=F, header=FALSE, row.names=1)
  ti=road_tissues[tissue_list,]
  rownames(mat)=ti
  
  # ###abbreviated names
  # road_tissues=read.csv("Roadmap_metadata.csv", stringsAsFactor=F, check.names=F, header=TRUE,row.names=1)
  # ti=road_tissues[tissue_list,]
  # rownames(mat)=ti$name
  
color=colorRampPalette(c("dodgerblue3", "white", "orangered3"))(50)
#color=colorRampPalette(c( "white", "dodgerblue3"))(50)
  
  pdf(paste( method_folder , "/",  names[i], ".heatmap_jaccard_only_enhancers_mean_center.pdf", sep=""), width=7, height=20)
  pheatmap(mat, show_rownames=TRUE,cluster_rows=F,
           annotation_col=ch.3, color = color,
           cellwidth = 8, cellheight = 8, annotation_colors=anno_colors, annotation_legend=FALSE,
           fontsize=9, border_color=NA, breaks=seq(-abs(max(mat)), abs(max(mat)), length.out=50) )
  
  
    
    
  dev.off()
  }


Warning message in dir.create(method_folder):
"'enhancers-mean-center' already exists"

### Method 4: Mean-center normalize only promoters

In [31]:
method_folder="promoters-mean-center"
dir.create(method_folder)

promoters=c('1_TssA' ,'2_PromU', '3_PromD1', '4_PromD2' ,'22_PromP' ,'23_PromBiv')

for (i in 1:length(files)) {

########################
  file=paste( home, "output/jaccard_analysis_25_state", files[i], "jaccard.tsv", sep="/")
  mat=read.table(file, header=T , row.names=1, check.names=FALSE, stringsAsFactor=FALSE)
  x=mat
    
    colnames(mat)=rownames(ch.3)
    mat<-subset(mat, select=as.character(promoters))
    mat<-mat[order(rowMeans(mat), decreasing=T),]
    #mat=scale(mat, center = TRUE, scale = TRUE)

  ########################################################################
  ##### Heatmaps    ######################################################
  

  tissue_list=rownames(mat)  
  
  mat=mat[tissue_list,]
  
  road_tissues=read.csv("EIDlegend.csv", stringsAsFactor=F, check.names=F, header=FALSE, row.names=1)
  ti=road_tissues[tissue_list,]
  rownames(mat)=make.unique(ti)
  
  # ###abbreviated names
  # road_tissues=read.csv("Roadmap_metadata.csv", stringsAsFactor=F, check.names=F, header=TRUE,row.names=1)
  # ti=road_tissues[tissue_list,]
  # rownames(mat)=ti$name
  
color=colorRampPalette(c("dodgerblue3", "white", "orangered3"))(50)
#color=colorRampPalette(c( "white", "dodgerblue3"))(50)
  
  pdf(paste( method_folder , "/",  names[i], ".heatmap_jaccard_only_promoters_mean_center.pdf", sep=""), width=7, height=20)
  pheatmap(mat, show_rownames=TRUE,cluster_rows=F,
           annotation_col=ch.3, color = color,
           cellwidth = 8, cellheight = 8, annotation_colors=anno_colors, annotation_legend=FALSE,
           fontsize=9, border_color=NA, breaks=seq(-abs(max(mat)), abs(max(mat)), length.out=50) )
  
  
    
    
  dev.off()
  }


Warning message in dir.create(method_folder):
"'promoters-mean-center' already exists"

### Method 5: Mean centers all states, ordered by enhancers

In [32]:
method_folder="mean-center"

enhancers=c('10_TxEnh5', '11_TxEnh3', '12_TxEnhW' ,'13_EnhA1' ,'14_EnhA2', '15_EnhAF', '16_EnhW1' ,'17_EnhW2', '18_EnhAc')


for (i in 1:length(files)) {

########################
  file=paste( home, "output/jaccard_analysis_25_state", files[i], "jaccard.tsv", sep="/")
  mat=read.table(file, header=T , row.names=1, check.names=FALSE, stringsAsFactor=FALSE)
  
    colnames(mat)=rownames(ch.3)
    x<-subset(mat, select=as.character(enhancers))
    mat<-mat[order(rowMeans(x), decreasing=T),]
    mat=scale(mat, center = TRUE, scale = FALSE)

  ########################################################################
  ##### Heatmaps    ######################################################
  

  tissue_list=rownames(mat)  
  
  mat=mat[tissue_list,]
  
  road_tissues=read.csv("EIDlegend.csv", stringsAsFactor=F, check.names=F, header=FALSE, row.names=1)
  ti=road_tissues[tissue_list,]
  rownames(mat)=ti
  
  # ###abbreviated names
  # road_tissues=read.csv("Roadmap_metadata.csv", stringsAsFactor=F, check.names=F, header=TRUE,row.names=1)
  # ti=road_tissues[tissue_list,]
  # rownames(mat)=ti$name
  
color=colorRampPalette(c("dodgerblue3", "white", "orangered3"))(50)

  
   pdf(paste( method_folder , "/", names[i], ".heatmap_jaccard_all_tissues_mean_center_ordered_by_enhancers.pdf", sep=""), width=10, height=20)
  pheatmap(mat, show_rownames=TRUE,cluster_rows=F,
           annotation_col=ch.3, color = color,
           cellwidth = 8, cellheight = 8, annotation_colors=anno_colors, annotation_legend=FALSE,
           fontsize=9, border_color=NA, breaks=seq(-abs(max(mat)), abs(max(mat)), length.out=50) )
  
  
    
    
  dev.off()
  }


### Method 6: Mean-center normalize  promoters and enhancers

In [33]:
method_folder="enhancers-and-promoters-mean-center"
dir.create(method_folder)


enhancers=c('2_PromU', '3_PromD1', '4_PromD2' ,'22_PromP' ,'23_PromBiv',
            '13_EnhA1' ,'14_EnhA2', '15_EnhAF' )

for (i in 1:length(files)) {

########################
  file=paste( home, "output/jaccard_analysis_25_state", files[i], "jaccard.tsv", sep="/")
  mat=read.table(file, header=T , row.names=1, check.names=FALSE, stringsAsFactor=FALSE)
  x=mat
    
    colnames(mat)=rownames(ch.3)
    mat<-subset(mat, select=as.character(enhancers))
    mat<-mat[order(rowMeans(mat), decreasing=T),]
    mat=scale(mat, center = TRUE, scale = FALSE)

  ########################################################################
  ##### Heatmaps    ######################################################
  

  tissue_list=rownames(mat)  
  
  mat=mat[tissue_list,]
  
  road_tissues=read.csv("EIDlegend.csv", stringsAsFactor=F, check.names=F, header=FALSE, row.names=1)
  ti=road_tissues[tissue_list,]
  rownames(mat)=ti
  
  # ###abbreviated names
  # road_tissues=read.csv("Roadmap_metadata.csv", stringsAsFactor=F, check.names=F, header=TRUE,row.names=1)
  # ti=road_tissues[tissue_list,]
  # rownames(mat)=ti$name
  
color=colorRampPalette(c("dodgerblue3", "white", "orangered3"))(50)
#color=colorRampPalette(c( "white", "dodgerblue3"))(50)
  
  pdf(paste( method_folder , "/",  names[i], ".heatmap_jaccard_enhancers_and_promoters_mean_center.pdf", sep=""), width=7, height=20)
  pheatmap(mat, show_rownames=TRUE,cluster_rows=F,
           annotation_col=ch.3, color = color,
           cellwidth = 8, cellheight = 8, annotation_colors=anno_colors, annotation_legend=FALSE,
           fontsize=9, border_color=NA, breaks=seq(-abs(max(mat)), abs(max(mat)), length.out=50) )
  
  
    
    
  dev.off()
  }


Warning message in dir.create(method_folder):
"'enhancers-and-promoters-mean-center' already exists"